# Car Price Predictor - Feature Engineering
### Siraj Ali

**Dataset Location**: https://www.kaggle.com/CooperUnion/cardataset

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Read in the dataset:

In [3]:
df = pd.read_csv('../data.csv')

## Handle duplicates and nulls:

In [4]:
dups = df[df.duplicated()]
print(dups.shape)

(715, 16)


In [5]:
df_dedup = df.drop_duplicates()
print(df_dedup.shape)

(11199, 16)


In [6]:
df_dedup.isnull().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3376
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64



`Engine Fuel Type`, `Engine HP`, `Engine Cylinders`, `Number of Doors`, and `Market Category` all have some null values. All of the mentioned columns, except for `Market Category`, have very few nulls. Let's take a closer look to see how to handle these:

### `Engine Fuel Type`

There are only 3 records will null here, so we can quickly check what the value should be.

It appears to specifically be missing only for the 2004 Suzuki Verona. From looking at the other records of the same generation of Suzuki Verona, we can find that that engine too regular unleaded fuel. This is confirmed by a quick search on the Internet.

In [7]:
df_dedup[df_dedup['Engine Fuel Type'].isna()]

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
11321,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,17199
11322,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,20199
11323,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,18499


In [8]:
df_dedup.query('Make == "Suzuki" and Model == "Verona"')

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
11321,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,17199
11322,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,20199
11323,Suzuki,Verona,2004,NaN,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,18499
11324,Suzuki,Verona,2005,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,18,481,19349
11325,Suzuki,Verona,2005,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,18,481,21049
11326,Suzuki,Verona,2005,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,18,481,17549
11327,Suzuki,Verona,2005,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,18,481,20549
11328,Suzuki,Verona,2006,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,20299
11329,Suzuki,Verona,2006,regular unleaded,155.0,6.0,AUTOMATIC,front wheel drive,4.0,NaN,Midsize,Sedan,25,17,481,18299


In [9]:
df_xform = df_dedup.copy()
df_xform['Engine Fuel Type'] = df_xform['Engine Fuel Type'].fillna('regular unleaded')

### `Engine HP`

There are 69 records that have missing horsepower numbers. After looking further, these are comprised of 16 unique vehicles (based on make, model, engine, and driven wheels). We can easily look up the missing figures here, but to save time, we can impute by filling in the mean of the corresponding car manufacturer (make).

Note: All Tesla records have null horsepower. Since there are many Model S variants, and we cannot impute the average, we'll drop these remaining rows.

In [10]:
grpby_cols = ['Make', 'Model', 'Engine Cylinders', 'Driven_Wheels']

null_hp_df = df_xform[df_xform['Engine HP'].isna()].groupby(grpby_cols).size().reset_index(name='count')
null_hp_df

,Make,Model,Engine Cylinders,Driven_Wheels,count
0,Chevrolet,Impala,6.0,front wheel drive,6
1,FIAT,500e,0.0,front wheel drive,3
2,Ford,Escape,4.0,all wheel drive,2
3,Ford,Escape,4.0,front wheel drive,2
4,Ford,Focus,0.0,front wheel drive,3
5,Ford,Freestar,6.0,front wheel drive,6
6,Honda,Fit EV,0.0,front wheel drive,2
7,Kia,Soul EV,0.0,front wheel drive,5
8,Lincoln,Continental,6.0,all wheel drive,2
9,Lincoln,Continental,6.0,front wheel drive,2


In [11]:
mean_hp = df_xform.merge(null_hp_df[['Make']], on=['Make'], how='inner').groupby(['Make'])['Engine HP'].mean()
mean_hp = pd.DataFrame(mean_hp)
mean_hp

,Engine HP
Make,
Chevrolet,249.483751
FIAT,143.559322
Ford,249.692118
Honda,196.772622
Kia,207.558036
Lincoln,286.125000
Mercedes-Benz,353.500000
Nissan,241.375231
Tesla,NaN


In [12]:
df_xform['Engine HP'] = df_xform['Engine HP'].fillna(df_xform['Make'].map(mean_hp['Engine HP']))

In [13]:
df_xform = df_xform.dropna(axis=0, subset=['Engine HP'])

## Engine Cylinders

The records with no indicated number of cylinders are either fully electric vehicles, or vehicles with rotary engines (the Mazda RX-7 and RX-8). So we will impute the nulls with zeroes.

In [14]:
df_xform[df_xform['Engine Cylinders'].isna()].groupby(['Make', 'Model']).size().reset_index(name='count')

,Make,Model,count
0,Chevrolet,Bolt EV,2
1,Mazda,RX-7,3
2,Mazda,RX-8,17
3,Mitsubishi,i-MiEV,2
4,Toyota,RAV4 EV,1
5,Volkswagen,e-Golf,4


In [15]:
df_xform['Engine Cylinders'] = df_xform['Engine Cylinders'].fillna(0)

## `Number of Doors`

There is only 1 record without an indicated number of doors, and it's the Ferrari FF. The FF is technically a 3-door shooting brake, since the rear hatch is counted as a door.

In [16]:
df_xform['Number of Doors'] = df_xform['Number of Doors'].fillna(3)

In [17]:
df_xform['Market Category'].unique()

array(['Factory Tuner,Luxury,High-Performance', 'Luxury,Performance',
       'Luxury,High-Performance', 'Luxury', 'Performance', 'Flex Fuel',
       'Flex Fuel,Performance', nan, 'Hatchback',
       'Hatchback,Luxury,Performance', 'Hatchback,Luxury',
       'Luxury,High-Performance,Hybrid', 'Diesel,Luxury',
       'Hatchback,Performance', 'Hatchback,Factory Tuner,Performance',
       'High-Performance', 'Factory Tuner,High-Performance',
       'Exotic,High-Performance', 'Exotic,Factory Tuner,High-Performance',
       'Factory Tuner,Performance', 'Crossover', 'Exotic,Luxury',
       'Exotic,Luxury,High-Performance', 'Exotic,Luxury,Performance',
       'Factory Tuner,Luxury,Performance', 'Flex Fuel,Luxury',
       'Crossover,Luxury', 'Hatchback,Factory Tuner,Luxury,Performance',
       'Crossover,Hatchback', 'Hybrid', 'Luxury,Performance,Hybrid',
       'Crossover,Luxury,Performance,Hybrid',
       'Crossover,Luxury,Performance',
       'Exotic,Factory Tuner,Luxury,High-Performance',
   

## `Market Category`

In [18]:
all_categories = set()
for market_cat in df_xform['Market Category'].dropna().unique():
    for c in market_cat.split(','):
        all_categories.add(c)

all_categories

{'Crossover',
 'Diesel',
 'Exotic',
 'Factory Tuner',
 'Flex Fuel',
 'Hatchback',
 'High-Performance',
 'Hybrid',
 'Luxury',
 'Performance'}

This field appears to have a list of one or many (or none) identifiers for each vehicle.
Let's parse this field and create new booleans indicators for each identifier.

In [19]:
def parse_market_category(mc):
    cat_dict = dict.fromkeys(all_categories, 0)
    if mc == mc: # account for nan
        for c in mc.split(','):
            cat_dict[c] = 1
    return cat_dict

In [20]:
market_category_df = df_xform['Market Category'].apply(parse_market_category).apply(pd.Series)

In [21]:
df_xform = pd.merge(df_xform, market_category_df, left_index=True, right_index=True)

In [22]:
df_xform.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,...,Hybrid,Crossover,High-Performance,Hatchback,Flex Fuel,Exotic,Factory Tuner,Diesel,Luxury,Performance
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",...,0,0,1,0,0,0,1,0,1,0
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",...,0,0,0,0,0,0,0,0,1,1
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",...,0,0,1,0,0,0,0,0,1,0
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",...,0,0,0,0,0,0,0,0,1,1
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,...,0,0,0,0,0,0,0,0,1,0


## Handle outliers

There are a few exotics/hypercars that cost over $1 million. Let's just drop these for simplicity, since these are outliers relative to the rest of the data.

In [28]:
df_xform[df_xform['MSRP'] > 1000000].shape[0]

6

In [29]:
df_xform = df_xform[df_xform['MSRP'] < 1000000]

## Convert year to age

This is more interpretable, though the information is the same.

In [30]:
df_xform.loc[:, 'Age'] = df_xform['Year'].apply(lambda x: 2017 - x)

## Drop unnecessary columns

In [31]:
df_xform = df_xform.drop(['Model', 'Market Category', 'Year'], axis=1)

## One-hot encode categorical columns

In [32]:
df_xform.iloc[0]

Make                                         BMW
Engine Fuel Type     premium unleaded (required)
Engine HP                                    335
Engine Cylinders                               6
Transmission Type                         MANUAL
Driven_Wheels                   rear wheel drive
Number of Doors                                2
Vehicle Size                             Compact
Vehicle Style                              Coupe
highway MPG                                   26
city mpg                                      19
Popularity                                  3916
MSRP                                       46135
Hybrid                                         0
Crossover                                      0
High-Performance                               1
Hatchback                                      0
Flex Fuel                                      0
Exotic                                         0
Factory Tuner                                  1
Diesel              

In [33]:
categorical_columns = [
    'Make',
    'Engine Fuel Type',
    'Transmission Type',
    'Driven_Wheels',
    'Vehicle Size',
    'Vehicle Style'
]

In [34]:
df_xform_final = pd.get_dummies(df_xform, columns=categorical_columns)

## Save transformed data to disk

In [35]:
df_xform_final.to_parquet('../data_xform.parquet', engine='pyarrow')